In [1]:
import pdblp
import pandas as pd
from datetime import date
import datetime
import sys, getopt

In [2]:
con = pdblp.BCon(port=8194, timeout=5000)
con.start()

In [6]:
today = '20' + date.today().strftime('%y%m%d')
df = con.bdh('SPY US Equity', flds=['PX_LAST', 'PX_OPEN', 'PX_HIGH', 'PX_LOW', 'MOV_AVG_200D', 'MOV_AVG_5D', 'MOV_AVG_50D'], start_date='20000101', end_date=today)

c:\ProgramData\Anaconda3\envs\blp_env\Lib\site-packages\pdblp\pdblp.py:278: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'date'] = pd.to_datetime(df.loc[:, 'date'])


In [7]:
df

ticker     SPY US Equity                                             \
field            PX_LAST   PX_OPEN   PX_HIGH    PX_LOW MOV_AVG_200D   
date                                                                  
2000-01-03      145.4375  148.2500  148.2500  143.8750     134.9281   
2000-01-04      139.7500  143.5313  144.0625  139.6406     134.9771   
2000-01-05      140.0000  139.9375  141.5313  137.2500     135.0462   
2000-01-06      137.7500  139.6250  141.5000  137.7500     135.1004   
2000-01-07      145.7500  140.3125  145.7500  140.0625     135.1817   
...                  ...       ...       ...       ...          ...   
2022-12-05      399.5900  403.9500  404.9300  398.1700     403.5566   
2022-12-06      393.8300  399.4200  399.9900  391.6400     403.3546   
2022-12-07      393.1600  392.9400  395.6400  391.9700     403.1726   
2022-12-08      396.2400  395.1400  397.3600  393.2684     403.0440   
2022-12-09      393.2700  394.9400  397.6200  393.1500     402.8688   

ticker                             
field      MOV_AVG_5D MOV_AVG_50D  
date                               
2000-01-03   146.3906    140.2524  
2000-01-04   145.1031    140.4455  
2000-01-05   143.7406    140.6568  
2000-01-06   141.9625    140.8555  
2000-01-07   141.7375    141.1680  
...               ...         ...  
2022-12-05   403.3580    380.9648  
2022-12-06   403.0780    381.5552  
2022-12-07   400.1740    382.1508  
2022-12-08   397.9460    382.6650  
2022-12-09   395.2180    383.2746  

[5773 rows x 7 columns]

In [8]:
window_size = 7
predictions = []
oneHotPreds = []

for i in range(len(df.values) - 1):
    row1 = df.values[i]
    row2 = df.values[i + 1]
    priceTomorrow = row2[1]
    priceToday = row1[0]

    if priceTomorrow > priceToday:
        predictions.append('UP')
        oneHotPreds.append(1)
    else:
        predictions.append('DOWN')
        oneHotPreds.append(0)

predictions.append('X')
oneHotPreds.append('X')
df['Predictions'] = predictions
df['OneHotPred'] = oneHotPreds


In [9]:
df

ticker     SPY US Equity                                             \
field            PX_LAST   PX_OPEN   PX_HIGH    PX_LOW MOV_AVG_200D   
date                                                                  
2000-01-03      145.4375  148.2500  148.2500  143.8750     134.9281   
2000-01-04      139.7500  143.5313  144.0625  139.6406     134.9771   
2000-01-05      140.0000  139.9375  141.5313  137.2500     135.0462   
2000-01-06      137.7500  139.6250  141.5000  137.7500     135.1004   
2000-01-07      145.7500  140.3125  145.7500  140.0625     135.1817   
...                  ...       ...       ...       ...          ...   
2022-12-05      399.5900  403.9500  404.9300  398.1700     403.5566   
2022-12-06      393.8300  399.4200  399.9900  391.6400     403.3546   
2022-12-07      393.1600  392.9400  395.6400  391.9700     403.1726   
2022-12-08      396.2400  395.1400  397.3600  393.2684     403.0440   
2022-12-09      393.2700  394.9400  397.6200  393.1500     402.8688   

ticker                            Predictions OneHotPred  
field      MOV_AVG_5D MOV_AVG_50D                         
date                                                      
2000-01-03   146.3906    140.2524        DOWN          0  
2000-01-04   145.1031    140.4455          UP          1  
2000-01-05   143.7406    140.6568        DOWN          0  
2000-01-06   141.9625    140.8555          UP          1  
2000-01-07   141.7375    141.1680          UP          1  
...               ...         ...         ...        ...  
2022-12-05   403.3580    380.9648        DOWN          0  
2022-12-06   403.0780    381.5552        DOWN          0  
2022-12-07   400.1740    382.1508          UP          1  
2022-12-08   397.9460    382.6650        DOWN          0  
2022-12-09   395.2180    383.2746           X          X  

[5773 rows x 9 columns]

In [12]:
from util import loadTickers
import random 
tickers = loadTickers()
tickers.pop(0)
tickers.pop(0)
random.shuffle(tickers)
tickers.insert(0, 'SPY US Equity')

for k, ticker in enumerate(tickers):
    if k % 25 == 0:
        print('PROCESSED', k)
    
    df = con.bdh(ticker, ['PX_LAST', 'PX_OPEN', 'MOV_AVG_200D', 'MOV_AVG_5D', 'MOV_AVG_50D'], start_date='20000101', end_date=today)
    predictions = []
    oneHotPreds = []
    priceTomorrows = []

    for i in range(len(df.values) - 1):
        row1 = df.values[i]
        row2 = df.values[i + 1]
        priceTomorrow = row2[0]
        priceToday = row1[0]
        priceTomorrows.append(priceTomorrow)

        if priceTomorrow > priceToday:
            predictions.append('UP')
            oneHotPreds.append(1)
        else:
            predictions.append('DOWN')
            oneHotPreds.append(0)

    predictions.append('X')
    oneHotPreds.append('X')
    priceTomorrows.append('X')

    df['Close Tomorrow'] = priceTomorrows

    fileName = '.\\newdata\\' + ticker.split(' ')[0] + '.csv'
    df.to_csv(fileName)

PROCESSED 0


KeyboardInterrupt: 